In [87]:
import findspark
findspark.init()

from pyspark.mllib.recommendation import ALS, Rating
from pyspark import SparkContext, SQLContext
sc = SparkContext("local", "test")
sqlContext = SQLContext(sc)

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=test, master=local) created by __init__ at <ipython-input-1-1b8b48b575e7>:6 

In [ ]:
TRAIN_FILE = "./debug_data/ratings-train-debug.dat/"
VALIDATION_FILE = "./data/ratings-validation.dat/"
TEST_FILE = "./data/ratings-test.dat/"

In [ ]:
def prepare_data(data):
    return (
        data
        .map(lambda l: l.split(','))
        .map(lambda l: Rating(int(l[0]), int(l[1]), float(l[2])))
    )  

In [ ]:
def prepare_data2(data):
    return (
        data
        .map(lambda l: l.split('::'))
        .map(lambda l: Rating(int(l[0]), int(l[1]), float(l[2])))
    )  

In [ ]:
# Load and parse the data
ratings_train_text = sc.textFile(TRAIN_FILE)
ratings_train = prepare_data(ratings_train_text)
ratings_train.take(10)

In [ ]:
ratings_validation_text = sc.textFile(VALIDATION_FILE)
ratings_validation = prepare_data2(ratings_validation_text)
ratings_validation.take(10)

In [ ]:
ratings_test_text = sc.textFile(TEST_FILE)
ratings_test = prepare_data2(ratings_validation_text)
ratings_test.take(10)

#### Calculate the general mean u for all ratings

In [ ]:
global_mean = ratings_train.map(lambda r: (r[2])).mean()

In [ ]:
global_mean

##### calculate item-specific bias, according to the paper we referenced, for each item i, its bias is equal to the summation of difference between all ratings of to the same item and global mean and then the result is divided by the sum of a regulation parameter and the quantity of the ratings.


In [ ]:
#convert training data to dataframe with attribute
df = sqlContext.createDataFrame(ratings_train, ['userId', 'movieId', 'ratings'])

In [ ]:
#sort the data by movie
df_orderByMovie = df.orderBy(df.movieId)

In [ ]:
#group the movie and count each movie
movie_count = df_orderByMovie.groupBy(df_orderByMovie.movieId).count()

In [ ]:
#calculate the sum of the ratings of each movie
sum_byMovie = df_orderByMovie.groupBy(['movieId']).sum()

In [ ]:
#drop some unrelated column
drop_column1 = sum_byMovie.drop(sum_byMovie[1])
final_drop = drop_column1.drop(drop_column1[1])

In [ ]:
#join the sum of count and sum of rating for each movie
movie_sorted = movie_count.join(final_drop, "movieId")

In [ ]:
#sorted the dataset by each movie
new_movie_sorted = movie_sorted.orderBy(movie_sorted.movieId)

In [ ]:
#calculate item specific bias
item_bias = new_movie_sorted.map(lambda r: [r[0], (r[2] - r[1]*global_mean)/(25+r[1])])

In [ ]:
new_item_bias = sqlContext.createDataFrame(item_bias, ['movieId', 'item_bias'])

#### Caculate the user-specific bias

In [ ]:
#order the training set by user
df_orderByUser = df.orderBy(df.userId)

In [ ]:
#join the item bias dataset to with the same movieId
contain_itemBias = df_orderByUser.join(new_item_bias, "movieId")

In [ ]:
#sorted the dataset by user
sorted_byUser = contain_itemBias.orderBy(['userId'])

In [ ]:
#calculate the numerical part of item specific bais
subtraction = sorted_byUser.map(lambda r: [r[1], r[2] - global_mean - r[3]])

In [ ]:
user_bias_part1 = sqlContext.createDataFrame(subtraction, ['userId', 'subtraction'])

In [ ]:
sum_byUser = user_bias_part1.groupBy(['userId']).sum()

In [ ]:
#count the user 
sum_UserCollect = user_bias_part1.groupBy(['userId']).count()

In [ ]:
#order the data set by user
ordered_sum_UserCollect = sum_UserCollect.orderBy(sum_UserCollect.userId)

In [ ]:
drop_column2 = sum_byUser.drop(sum_byUser[1])

In [ ]:
final_drop2 = drop_column2.orderBy(drop_column2.userId)

In [ ]:
user_bias_table = final_drop2.join(ordered_sum_UserCollect, 'userId')

In [ ]:
ordered_userBiaTable = user_bias_table.orderBy(user_bias_table.userId)

In [ ]:
user_bias = ordered_userBiaTable.map(lambda r: [r[0], r[1]/(10+r[2])])

In [ ]:
user_specific_bias = sqlContext.createDataFrame(user_bias, ['userId', 'user_bias'])

In [ ]:
merge1 = df_orderByUser.join(user_specific_bias, 'userId')

In [ ]:
merge2 = merge1.join(new_item_bias, 'movieId')

In [ ]:
new_ratings_train = merge2.map(lambda r: [r[0], r[1], r[2] - r[3] - r[4]])

In [ ]:
temp = sqlContext.createDataFrame(new_ratings_train, ['movieId', 'userId', 'new_ratings'])

In [ ]:
final_new_ratings_train = temp.orderBy(temp.userId)

In [ ]:
final_new_ratings_train.take(10)

In [ ]:
#now, we perform the same procedure as task1
#first, we sort the data by timestamp. 
new_ratings_byTime = final_new_ratings_train.join(df, ['userId', 'movieId'])

In [ ]:
#example of dataset
new_ratings_byTime.take(20)

In [ ]:
new_ratings_byTime = new_ratings_byTime.drop(new_ratings_byTime[3])

In [ ]:
def prepare_validation(validation):
    return validation.map(lambda p: (p[0], p[1]))

In [ ]:
import math

In [ ]:
# Evaluate the model on training data

def train_evaluate_als(train, validation, rank, iterations_num, lambda_val):
    print("Validation %s" % validation.take(100))
    model = ALS.train(train, rank, iterations_num, lambda_val)
    predictions = model.predictAll(prepare_validation(validation)).map(lambda r: ((r[0], r[1]), r[2]))
    print("Predictions %s" % predictions.take(100))
    ratesAndPreds = validation.map(lambda r: ((r[0], r[1]), r[2])).join(predictions)
    print("ratesAndPreds %s" % ratesAndPreds.take(100))
    MSE = ratesAndPreds.map(lambda r: (r[1][0] - r[1][1])**2).mean()
    RMSE = math.sqrt(MSE)
    return MSE, RMSE

In [ ]:
ranks = [10, 20, 30, 40, 50]
lambda_values = [0.01,0.1,1.0,10.0]
ITERATIONS = 10

In [ ]:
def report_mse_results(rank, lambda_value, mse, rmse):
    print("Rank=%d, Lambda=%0.2f, MSE=%s, RMSE=%s" % (rank, lambda_value, mse, rmse))

In [ ]:
def evaluate_parameters(train, validation, ranks, lambda_values):
    for r in ranks:
        for l in lambda_values:
            mse, rmse = train_evaluate_als(new_ratings_byTime, validation, r, ITERATIONS, l)
            report_mse_results(r, l, mse, rmse)

In [ ]:
evaluate_parameters(new_ratings_byTime, ratings_validation, ranks, lambda_values)